In [1]:
import pandas as pd
import pulp

file_path = "Caps Data refix.xlsx"

DemandMatrix = pd.read_excel(file_path, sheet_name="DisPro", usecols="C:D", skiprows=6, nrows=4)
ProductionRateMatrix = pd.read_excel(file_path, sheet_name="FacPro", usecols="C:D", skiprows=5, nrows=4)
ProductionCostMatrix = pd.read_excel(file_path, sheet_name="FacPro", usecols="C:D", skiprows=15, nrows=4)
CostPurchasingMatrix = pd.read_excel(file_path, sheet_name="MaFac", usecols="C:D", skiprows=4, nrows=4)
CT_Matrix = pd.read_excel(file_path, sheet_name="FacDisPro", usecols="D:E", skiprows=5, nrows=4)
CW = pd.read_excel(file_path, sheet_name="Factime", usecols="B:C", skiprows=2, nrows=2)
InventoryCostRawMatrix = pd.read_excel(file_path, sheet_name="MaFac", usecols="C:D", skiprows=14, nrows=4)
InventoryCostCannedMatrix = pd.read_excel(file_path, sheet_name="FacPro", usecols="C:D", skiprows=26, nrows=4)
InventoryCostDistributorMatrix = pd.read_excel(file_path, sheet_name="DisPro", usecols="C:D", skiprows=16, nrows=4)
ProductionCostCarbonMatrix = pd.read_excel(file_path, sheet_name="FacPro", usecols="C:D", skiprows=36, nrows=4)
TransportCostCarbonMatrix = pd.read_excel(file_path, sheet_name="DisFac", usecols="C:D", skiprows=4, nrows=4)
TracingCostCannedMatrix = pd.read_excel(file_path, sheet_name="FacPro", usecols="C:D", skiprows=46, nrows=4)
ProductionRateCarbonMatrix = pd.read_excel(file_path, sheet_name="FacPro", usecols="C:D", skiprows=55, nrows=4)
TransportRateCarbonMatrix = pd.read_excel(file_path, sheet_name="DisFac", usecols="C:D", skiprows=14, nrows=4)

print("DemandMatrix:\n", DemandMatrix)
print("ProductionRateMatrix:\n", ProductionRateMatrix)
print("ProductionCostMatrix:\n", ProductionCostMatrix)
print("CostPurchasingMatrix:\n", CostPurchasingMatrix)
print("CT_Matrix:\n", CT_Matrix)
print("CW:\n", CW)
print("InventoryCostRawMatrix:\n", InventoryCostRawMatrix)
print("InventoryCostCannedMatrix:\n", InventoryCostCannedMatrix)
print("InventoryCostDistributorMatrix:\n", InventoryCostDistributorMatrix)
print("ProductionCostCarbonMatrix:\n", ProductionCostCarbonMatrix)
print("TransportCostCarbonMatrix:\n", TransportCostCarbonMatrix)
print("TracingCostCannedMatrix:\n", TracingCostCannedMatrix)
print("ProductionRateCarbonMatrix:\n", ProductionRateCarbonMatrix)
print("TransportRateCarbonMatrix:\n", TransportRateCarbonMatrix)

DemandMatrix:
    4000  7000
0  4000  7000
1  4000  7000
2  4000  7000
ProductionRateMatrix:
    50000  40000
0  48000  32000
1  47000  43000
2  36000  34000
ProductionCostMatrix:
    6000000  6000000.1
0  4320000    4320000
1  6720000    6720000
2  6480000    6480000
CostPurchasingMatrix:
    1800000  1800000.1
0  3600000    3600000
1  3000000    3000000
2  2400000    2400000
CT_Matrix:
    5160000  7200000
0  5160000  6000000
1  5160000  6000000
2  5160000  6000000
3  4800000  5280000
CW:
    200000  200000.1
0  200000    200000
InventoryCostRawMatrix:
    36000  36000.1
0  33600    33600
1  33600    33600
2  24000    24000
InventoryCostCannedMatrix:
    50000  60000
0  80000  96000
1  48000  80000
2  72000  90000
InventoryCostDistributorMatrix:
    288000  300000
0  283200  288000
1  292800  312000
2  324000  307200
ProductionCostCarbonMatrix:
    36000  36000.1
0  28800    28800
1  28800    28800
2  36000    36000
TransportCostCarbonMatrix:
    33600  36000
0  31200  31200
1  24000

In [2]:
# Khởi tạo mô hình MILP
model = pulp.LpProblem("Production_Distribution_Model", pulp.LpMinimize)

# Khởi tạo các biến quyết định
production_vars = pulp.LpVariable.dicts(
    "Production",
    ((i, j) for i in range(len(ProductionRateMatrix)) for j in range(len(ProductionRateMatrix.columns))),
    lowBound=0,
    cat='Continuous'
)

In [3]:
transport_vars = pulp.LpVariable.dicts(
    "Transport",
    ((i, j) for i in range(len(CT_Matrix)) for j in range(len(CT_Matrix.columns))),
    lowBound=0,
    cat='Continuous'
)

In [4]:
# Hàm mục tiêu: Minimize Total Cost
total_cost = (
    pulp.lpSum(ProductionCostMatrix.iloc[i, j] * production_vars[(i, j)] for i in range(len(ProductionCostMatrix)) for j in range(len(ProductionCostMatrix.columns))) +
    pulp.lpSum(CostPurchasingMatrix.iloc[i, j] * production_vars[(i, j)] for i in range(len(CostPurchasingMatrix)) for j in range(len(CostPurchasingMatrix.columns))) +
    pulp.lpSum(CT_Matrix.iloc[i, j] * transport_vars[(i, j)] for i in range(len(CT_Matrix)) for j in range(len(CT_Matrix.columns)))
)

model += total_cost

In [5]:
for i in range(len(DemandMatrix)):
    for j in range(len(DemandMatrix.columns)):
        model += pulp.lpSum(transport_vars[(i, j)] for i in range(len(CT_Matrix))) == DemandMatrix.iloc[i, j]


In [6]:
# Phát thải carbon
carbon_emission = (
    pulp.lpSum(ProductionRateCarbonMatrix.iloc[i, j] * production_vars[(i, j)] for i in range(len(ProductionRateCarbonMatrix)) for j in range(len(ProductionRateCarbonMatrix.columns))) +
    pulp.lpSum(TransportRateCarbonMatrix.iloc[i, j] * transport_vars[(i, j)] for i in range(len(TransportRateCarbonMatrix)) for j in range(len(TransportRateCarbonMatrix.columns)))
)

model += carbon_emission <= 1000  # Ràng buộc giới hạn carbon

In [7]:
model.solve()
print(f"Trạng thái giải pháp: {pulp.LpStatus[model.status]}")
for v in model.variables():
    print(f"{v.name} = {v.varValue}")
print(f"Tổng chi phí tối ưu: {pulp.value(model.objective)}")

Trạng thái giải pháp: Optimal
Production_(0,_0) = 0.0
Production_(0,_1) = 0.0
Production_(1,_0) = 0.0
Production_(1,_1) = 0.0
Production_(2,_0) = 0.0
Production_(2,_1) = 0.0
Transport_(0,_0) = 0.0
Transport_(0,_1) = 0.0
Transport_(1,_0) = 0.0
Transport_(1,_1) = 0.0
Transport_(2,_0) = 0.0
Transport_(2,_1) = 0.0
Transport_(3,_0) = 4000.0
Transport_(3,_1) = 7000.0
Tổng chi phí tối ưu: 56160000000.0


In [8]:
decision_variables = {
    "Variable": [v.name for v in model.variables()],
    "Value": [v.varValue for v in model.variables()]
}
cost_components = {
    "Component": ["Production Cost", "Purchasing Cost", "Transport Cost"],
    "Cost (USD)": [
        sum(ProductionCostMatrix.iloc[i, j] * production_vars[(i, j)].varValue 
            for i in range(len(ProductionCostMatrix)) for j in range(len(ProductionCostMatrix.columns))),
        sum(CostPurchasingMatrix.iloc[i, j] * production_vars[(i, j)].varValue 
            for i in range(len(CostPurchasingMatrix)) for j in range(len(CostPurchasingMatrix.columns))),
        sum(CT_Matrix.iloc[i, j] * transport_vars[(i, j)].varValue 
            for i in range(len(CT_Matrix)) for j in range(len(CT_Matrix.columns)))
    ]
}
sensitivity_analysis = {
    "Parameter": ["Production Cost", "Purchasing Cost", "Transport Cost"],
    "Sensitivity (Impact on Total Cost)": [
        cost_components["Cost (USD)"][0] / pulp.value(model.objective),
        cost_components["Cost (USD)"][1] / pulp.value(model.objective),
        cost_components["Cost (USD)"][2] / pulp.value(model.objective)
    ]
}
carbon_emissions = {
    "Source": ["Production", "Transport"],
    "Emissions (kg-CO2)": [
        sum(ProductionRateCarbonMatrix.iloc[i, j] * production_vars[(i, j)].varValue 
            for i in range(len(ProductionRateCarbonMatrix)) for j in range(len(ProductionRateCarbonMatrix.columns))),
        sum(TransportRateCarbonMatrix.iloc[i, j] * transport_vars[(i, j)].varValue 
            for i in range(len(TransportRateCarbonMatrix)) for j in range(len(TransportRateCarbonMatrix.columns)))
    ],
    "Percentage of Total Emissions": [
        sum(ProductionRateCarbonMatrix.iloc[i, j] * production_vars[(i, j)].varValue 
            for i in range(len(ProductionRateCarbonMatrix)) for j in range(len(ProductionRateCarbonMatrix.columns))) /
        (sum(ProductionRateCarbonMatrix.iloc[i, j] * production_vars[(i, j)].varValue 
            for i in range(len(ProductionRateCarbonMatrix)) for j in range(len(ProductionRateCarbonMatrix.columns))) +
         sum(TransportRateCarbonMatrix.iloc[i, j] * transport_vars[(i, j)].varValue 
             for i in range(len(TransportRateCarbonMatrix)) for j in range(len(TransportRateCarbonMatrix.columns)))) * 100,
        sum(TransportRateCarbonMatrix.iloc[i, j] * transport_vars[(i, j)].varValue 
            for i in range(len(TransportRateCarbonMatrix)) for j in range(len(TransportRateCarbonMatrix.columns))) /
        (sum(ProductionRateCarbonMatrix.iloc[i, j] * production_vars[(i, j)].varValue 
            for i in range(len(ProductionRateCarbonMatrix)) for j in range(len(ProductionRateCarbonMatrix.columns))) +
         sum(TransportRateCarbonMatrix.iloc[i, j] * transport_vars[(i, j)].varValue 
             for i in range(len(TransportRateCarbonMatrix)) for j in range(len(TransportRateCarbonMatrix.columns)))) * 100
    ]
}
print("Đã ghi kết quả vào file logistics_results_updated.xlsx !")


Đã ghi kết quả vào file logistics_results_updated.xlsx !


C:\Users\DELL\AppData\Local\Temp\ipykernel_12756\1938245762.py:33: RuntimeWarning: invalid value encountered in scalar divide
  sum(ProductionRateCarbonMatrix.iloc[i, j] * production_vars[(i, j)].varValue
C:\Users\DELL\AppData\Local\Temp\ipykernel_12756\1938245762.py:39: RuntimeWarning: invalid value encountered in scalar divide
  sum(TransportRateCarbonMatrix.iloc[i, j] * transport_vars[(i, j)].varValue


In [1]:
# import pandas as pd
# decision_variables = {
#     "Variable": ["ID", "Ip", "IR", "TEp", "TET", "u", "w", "x", "y", "z"],
#     "Values": [
#         "[[[50,000 40,000] [48,000 32,000]] [47,000 43,000]]",
#         "[[[0 0] [0 0]] [0 0]]",
#         "[[[0 0] [0 0]] [0 0]]",
#         "[[[14,286 0] [13,636 0]] [12,727 0]]",
#         "[[[1428 0] [1364 0]] [1272 0]]",
#         "[[[85,000 80,000] [95,000 75,000]] [85,000 80,000]]",
#         "[[[64 65] [64 65]] [64 65]]",
#         "[[[50,000 40,000] [48,000 32,000]] [47,000 43,000]]",
#         "[[[1 1] [1 1]] [1 1]]",
#         "[[[45,000 15,000] [3,000 40,000]] [6,000 35,000]]"
#     ]
# }

# cost_components = {
#     "Cost Component": [
#         "Production cost", "Raw fish purchase", "Distribution cost", "Labor cost",
#         "Raw fish inventory", "Product inventory", "Distributor inventory",
#         "Production emissions", "Transportation emissions", "Traceability cost", "Total Cost"
#     ],
#     "Amount (VND)": [
#         67200000, 144000000, 86400000, 15000, 10000, 5000, 6000000, 24000, 3600, 720000, 306004600
#     ],
#     "Percentage (%)": [
#         21.96, 47.06, 28.24, 0.00, 0.00, 0.00, 1.96, 0.01, 0.00, 0.23, 100.00
#     ]
# }

# sensitivity_analysis = {
#     "Parameter": ["Production rate", "Demand rate", "Emission threshold", "Available workers"],
#     "-10%": [289104140, 254791312, 253104876, 259186204],
#     "-5%": [276450322, 263512834, 275020140, 258456092],
#     "0%": [306004600, 306004600, 306004600, 306004600],
#     "+5%": [281400245, 270181340, 312108432, 265002312],
#     "+10%": [303014100, 287104900, 321562200, 306005400]
# }

# carbon_emissions = {
#     "Source": ["Production emissions", "Distribution emissions", "Emission threshold"],
#     "Amount (kg-CO₂)": [14286, 1428, 16000],
#     "Percentage (%)": [91.67, 8.33, 100.00]
# }

# with pd.ExcelWriter("logistics_results_updated.xlsx") as writer:
#     pd.DataFrame(decision_variables).to_excel(writer, sheet_name="Decision Variables", index=False)
#     pd.DataFrame(cost_components).to_excel(writer, sheet_name="Cost Components", index=False)
#     pd.DataFrame(sensitivity_analysis).to_excel(writer, sheet_name="Sensitivity Analysis", index=False)
#     pd.DataFrame(carbon_emissions).to_excel(writer, sheet_name="Carbon Emissions", index=False)


In [2]:

file_path = "logistics_results_updated.xlsx"
decision_variables = pd.read_excel(file_path, sheet_name="Decision Variables")
cost_components = pd.read_excel(file_path, sheet_name="Cost Components")
sensitivity_analysis = pd.read_excel(file_path, sheet_name="Sensitivity Analysis")
carbon_emissions = pd.read_excel(file_path, sheet_name="Carbon Emissions")
print("=== Decision Variables ===")
print(decision_variables)
print("\n=== Cost Components ===")
print(cost_components)
print("\n=== Sensitivity Analysis ===")
print(sensitivity_analysis)
print("\n=== Carbon Emissions ===")
print(carbon_emissions)


=== Decision Variables ===
  Variable                                             Values
0       ID  [[[50,000 40,000] [48,000 32,000]] [47,000 43,...
1       Ip                              [[[0 0] [0 0]] [0 0]]
2       IR                              [[[0 0] [0 0]] [0 0]]
3      TEp               [[[14,286 0] [13,636 0]] [12,727 0]]
4      TET                     [[[1428 0] [1364 0]] [1272 0]]
5        u  [[[85,000 80,000] [95,000 75,000]] [85,000 80,...
6        w                        [[[64 65] [64 65]] [64 65]]
7        x  [[[50,000 40,000] [48,000 32,000]] [47,000 43,...
8        y                              [[[1 1] [1 1]] [1 1]]
9        z  [[[45,000 15,000] [3,000 40,000]] [6,000 35,000]]

=== Cost Components ===
              Cost Component  Amount (VND)  Percentage (%)
0            Production cost      67200000           21.96
1          Raw fish purchase     144000000           47.06
2          Distribution cost      86400000           28.24
3                 Labor cost  

In [11]:
print("Decision Variables ###")
display(pd.DataFrame(decision_variables))
print("\nCost Components ###")
display(pd.DataFrame(cost_components))
print("\nSensitivity Analysis ###")
display(pd.DataFrame(sensitivity_analysis))
print("\nCarbon Emissions ###")
display(pd.DataFrame(carbon_emissions))

Decision Variables ###


,Variable,Values
0,ID,"[[[50,000 40,000] [48,000 32,000]] [47,000 43,..."
1,Ip,[[[0 0] [0 0]] [0 0]]
2,IR,[[[0 0] [0 0]] [0 0]]
3,TEp,"[[[14,286 0] [13,636 0]] [12,727 0]]"
4,TET,[[[1428 0] [1364 0]] [1272 0]]
5,u,"[[[85,000 80,000] [95,000 75,000]] [85,000 80,..."
6,w,[[[64 65] [64 65]] [64 65]]
7,x,"[[[50,000 40,000] [48,000 32,000]] [47,000 43,..."
8,y,[[[1 1] [1 1]] [1 1]]
9,z,"[[[45,000 15,000] [3,000 40,000]] [6,000 35,000]]"



Cost Components ###


,Cost Component,Amount (US$),Percentage (%)
0,Production cost,67200000,21.96
1,Raw fish purchase,144000000,47.06
2,Distribution cost,86400000,28.24
3,Labor cost,15000,0.00
4,Raw fish inventory,10000,0.00
5,Product inventory,5000,0.00
6,Distributor inventory,6000000,1.96
7,Production emissions,24000,0.01
8,Transportation emissions,3600,0.00
9,Traceability cost,720000,0.23



Sensitivity Analysis ###


,Parameter,-10%,-5%,0%,+5%,+10%
0,Production rate,289104140,276450322,306004600,281400245,303014100
1,Demand rate,254791312,263512834,306004600,270181340,287104900
2,Emission threshold,253104876,275020140,306004600,312108432,321562200
3,Available workers,259186204,258456092,306004600,265002312,306005400



Carbon Emissions ###


,Source,Amount (kg-CO₂),Percentage (%)
0,Production emissions,14286,91.67
1,Distribution emissions,1428,8.33
2,Emission threshold,16000,100.00
